In [2]:
using JuMP, Gurobi
using QPSReader
using QuadraticModels
using NLPModels

In [3]:

function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0,  name=name_pb)

end

function display_results_Gurobi(gurobi_model)
    # show results
    sol = get_solution(gurobi_model)
    println("soln = $(sol)")

    objv = get_objval(gurobi_model)
    println("objv = $(objv)")
end
    

function OptimizeGurobi(qpdata; name_pb="lp")
    QM = createQuadraticModel(qpdata, name_pb)
    SM = SlackModel(QM)
    
    env = Gurobi.Env()

    # set presolve to 0
    setparam!(env, "Presolve", 0)
    
    model = gurobi_model(env; name = "lp_01", f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)

     # run optimization
    optimize(model)
    return model
end



OptimizeGurobi (generic function with 1 method)

In [7]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [8]:
model = OptimizeGurobi(qpdata2)
display_results_Gurobi(model)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 27 rows, 51 columns and 102 nonzeros
Model fingerprint: 0x84b9128a
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [8e+01, 5e+02]
  RHS range        [4e+01, 4e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.8000000e+30   6.800000e+30   1.800000e+00      0s
      10   -4.6475314e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds
Optimal objective -4.647531429e+02
soln = [80.0, 25.5, 54.5, 84.80000000000001, 18.214285714285715, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.214285714285715, 0.0, 19.30714285714286, 500.0, 475.92, 24.08, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.9428571428572, 383.9428571428572, 0.0, 0.0, 80.0, 0.0, 18.214285714285715, -0.0, -0.0, -0.0, 500.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -378.46